In [1]:
import pandas as pd
import numpy as np

In [3]:
#Display option to see more columns. Usefull to see all the columns of Dummy variables for Machine Learning
pd.set_option('display.max_columns', 120)


#make the charts look better for displays with retina resolution 
%config InlineBackend.figure_format = 'retina'

In [13]:
#Read in milk collected data
df_MilkCollected = pd.read_csv('C:\\Project CA2\\Data\\milk products linear.csv')

In [14]:
df_MilkCollected.head(20)

,DATAFLOW,LAST UPDATE,freq,dairyprod,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:APRO_MK_COLA(1.0),30/11/22 23:00:00,A,D1110D,THS_T,AL,2017,57.36,p
1,ESTAT:APRO_MK_COLA(1.0),30/11/22 23:00:00,A,D1110D,THS_T,AL,2018,64.39,NaN
2,ESTAT:APRO_MK_COLA(1.0),30/11/22 23:00:00,A,D1110D,THS_T,AL,2019,56.82,NaN
3,ESTAT:APRO_MK_COLA(1.0),30/11/22 23:00:00,A,D1110D,THS_T,AL,2020,56.27,NaN
4,ESTAT:APRO_MK_COLA(1.0),30/11/22 23:00:00,A,D1110D,THS_T,AL,2021,58.94,NaN
5,ESTAT:APRO_MK_COLA(1.0),30/11/22 23:00:00,A,D1110D,THS_T,AT,1995,2290.00,NaN
6,ESTAT:APRO_MK_COLA(1.0),30/11/22 23:00:00,A,D1110D,THS_T,AT,1996,2343.00,NaN
7,ESTAT:APRO_MK_COLA(1.0),30/11/22 23:00:00,A,D1110D,THS_T,AT,1997,2420.00,NaN
8,ESTAT:APRO_MK_COLA(1.0),30/11/22 23:00:00,A,D1110D,THS_T,AT,1998,2430.00,NaN
9,ESTAT:APRO_MK_COLA(1.0),30/11/22 23:00:00,A,D1110D,THS_T,AT,1999,2535.00,NaN
